https://www.kaggle.com/anonamename/check-oof

In [1]:
!pwd
import sys

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext lab_black
sys.executable

/c/Users/81908/jupyter_notebook/pytorch_lightning_work/kaggle_Cassava/notebook/check_oof/check_oof


'C:\\Users\\81908\\Anaconda3\\envs\\lightning\\python.exe'

In [2]:
import json
import pandas as pd

ROOT_DIR = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\input\cassava-leaf-disease-classification"

with open(f"{ROOT_DIR}/label_num_to_disease_map.json", "r") as f:
    name_mapping = json.load(f)
name_mapping = {int(k): v for k, v in name_mapping.items()}

df = pd.read_csv(f"{ROOT_DIR}/train.csv")
df

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [3]:
# oof確認

import glob
import pathlib
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn

oof_list = []
cfms = []
ens_weights = []
model_names = []


def check_oof_SiNpcw(Y, pkl_path=""):
    name = str(pathlib.Path(pkl_path).name)
    Y_oof = pickle.load(open(f"{pkl_path}", "rb"))
    oof = accuracy_score(Y, Y_oof.values.argmax(1))
    cfm = confusion_matrix(Y, Y_oof.values.argmax(1))

    print(f"{name} oof:    \t{round(oof, 4)}")

    oof_list.append(Y_oof.values)
    cfms.append(cfm)
    ens_weights.append(oof)
    model_names.append(name)

    return Y_oof.values, oof


def check_oof_Fukuda(Y, pkl_path=""):
    name = str(pathlib.Path(pkl_path).name)
    Y_oof = pickle.load(open(f"{pkl_path}", "rb"))

    # 確信度に変更
    Y_oof = torch.from_numpy(Y_oof.values.astype(np.float32)).clone()
    Y_oof = torch.softmax(Y_oof, 1).detach().cpu().numpy()

    oof = accuracy_score(Y, Y_oof.argmax(1))
    cfm = confusion_matrix(Y, Y_oof.argmax(1))

    print(f"{name} oof:    \t{round(oof, 4)}")

    oof_list.append(Y_oof)
    cfms.append(cfm)
    ens_weights.append(oof)
    model_names.append(name)

    return Y_oof, oof


# --------- anonamename ---------
print("-" * 100)
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\check_oof\cassava-emsemble-v2_tta_oof\kaggle_upload_oof_tta"
for pkl_path in sorted(glob.glob(f"{m_dir}/*.pkl")):
    Y_oof, oof = check_oof_SiNpcw(df["label"].values, pkl_path=pkl_path)

# --------- SiNpcw ---------
print("-" * 100)
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassavapkl"
for pkl_path in sorted(glob.glob(f"{m_dir}/*.pkl")):
    Y_oof, oof = check_oof_SiNpcw(df["label"].values, pkl_path=pkl_path)

## --------- Shun Fukuda ---------
# print("-" * 100)
# m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\confidence20210207"
# for pkl_path in sorted(glob.glob(f"{m_dir}/*.pkl")):
#    Y_oof, oof = check_oof_Fukuda(df["label"].values, pkl_path=pkl_path)

print("-" * 100)
print("ens_weights:", ens_weights)

oof_ensemble = np.zeros((df.shape[0], 5))
for i, pre in enumerate(oof_list):
    pre = pre[:21397]
    oof_ensemble += (ens_weights[i] / sum(ens_weights)) * pre

print("-" * 100)
ens_oof = accuracy_score(df["label"].values, oof_ensemble.argmax(1))
print(f"ens oof:    \t{round(ens_oof, 4)}")

----------------------------------------------------------------------------------------------------
byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl oof:    	0.8942
deit_base_patch16_224_tta.pkl oof:    	0.8853
microsoftvisionresnet50_tta.pkl oof:    	0.8866
resnest101e_cleanlab_noise_cutmix_tta.pkl oof:    	0.8949
resnest101e_cleanlab_tta.pkl oof:    	0.8948
resnest101e_tta.pkl oof:    	0.8942
seresnext50_32x4d_fmix_tta.pkl oof:    	0.8834
tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl oof:    	0.8881
tf_efficientnet_b4_ns_fold10_tta.pkl oof:    	0.8921
tf_efficientnet_b4_ns_fold3_tta.pkl oof:    	0.8925
tf_efficientnet_b4_ns_tta.pkl oof:    	0.8924
vit_b16_224_fold10_tta.pkl oof:    	0.8861
vit_base_patch16_224.fit_tta.pkl oof:    	0.8792
vit_base_patch32_384.fit_tta.pkl oof:    	0.8831
----------------------------------------------------------------------------------------------------
22019613_efficientnet-b4.pkl oof:    	0.8964
22019613_tta3_efficientnet-b4.

# cfm

In [4]:
for i, m in enumerate(model_names):
    print(m, ":\t", np.diag(cfms[i]))

display(
    pd.DataFrame(
        {
            "model": model_names,
            "cls0": np.array([np.diag(_c)[0] for _c in cfms]),
            "cls1": np.array([np.diag(_c)[1] for _c in cfms]),
            "cls2": np.array([np.diag(_c)[2] for _c in cfms]),
            "cls3": np.array([np.diag(_c)[3] for _c in cfms]),
            "cls4": np.array([np.diag(_c)[4] for _c in cfms]),
            "sum": np.array([np.diag(_c).sum() for _c in cfms]),
        }
    ).style.background_gradient(  # .sort_values(by=["cls0"], ascending=False)
        cmap="Blues"
    )
)

byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl :	 [  719  1743  1993 12756  1922]
deit_base_patch16_224_tta.pkl :	 [  664  1721  1788 12832  1938]
microsoftvisionresnet50_tta.pkl :	 [  672  1703  1901 12701  1994]
resnest101e_cleanlab_noise_cutmix_tta.pkl :	 [  693  1770  1962 12734  1989]
resnest101e_cleanlab_tta.pkl :	 [  684  1764  1952 12775  1972]
resnest101e_tta.pkl :	 [  753  1740  1913 12762  1965]
seresnext50_32x4d_fmix_tta.pkl :	 [  738  1732  2025 12614  1794]
tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl :	 [  796  1734  1973 12608  1892]
tf_efficientnet_b4_ns_fold10_tta.pkl :	 [  719  1767  1991 12697  1914]
tf_efficientnet_b4_ns_fold3_tta.pkl :	 [  730  1777  1994 12752  1843]
tf_efficientnet_b4_ns_tta.pkl :	 [  745  1750  1954 12777  1869]
vit_b16_224_fold10_tta.pkl :	 [  669  1727  1811 12828  1924]
vit_base_patch16_224.fit_tta.pkl :	 [  614  1742  1760 12742  1955]
vit_base_patch32_384.fit_tta.pkl :	 [  657  1730  1753 12859  1896]
22019613_ef

,model,cls0,cls1,cls2,cls3,cls4,sum
0,byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl,719,1743,1993,12756,1922,19133
1,deit_base_patch16_224_tta.pkl,664,1721,1788,12832,1938,18943
2,microsoftvisionresnet50_tta.pkl,672,1703,1901,12701,1994,18971
3,resnest101e_cleanlab_noise_cutmix_tta.pkl,693,1770,1962,12734,1989,19148
4,resnest101e_cleanlab_tta.pkl,684,1764,1952,12775,1972,19147
5,resnest101e_tta.pkl,753,1740,1913,12762,1965,19133
6,seresnext50_32x4d_fmix_tta.pkl,738,1732,2025,12614,1794,18903
7,tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl,796,1734,1973,12608,1892,19003
8,tf_efficientnet_b4_ns_fold10_tta.pkl,719,1767,1991,12697,1914,19088
9,tf_efficientnet_b4_ns_fold3_tta.pkl,730,1777,1994,12752,1843,19096


# corr

In [5]:
corr = np.empty((len(model_names), len(model_names)))
corr = pd.DataFrame(corr, columns=model_names, index=model_names)

for i, row in enumerate(model_names):
    for j, column in enumerate(model_names):
        if i <= j:
            corr.loc[row, column] = 0
        else:
            _df = pd.DataFrame(oof_list[i])
            other = pd.DataFrame(oof_list[j])

            corr.loc[row, column] = _df.corrwith(other).mean()

corr.style.background_gradient(cmap="Blues", subset=model_names, vmax=1.0, vmin=0.0)

,byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl,deit_base_patch16_224_tta.pkl,microsoftvisionresnet50_tta.pkl,resnest101e_cleanlab_noise_cutmix_tta.pkl,resnest101e_cleanlab_tta.pkl,resnest101e_tta.pkl,seresnext50_32x4d_fmix_tta.pkl,tf_efficientnet_b4_ns_cleanlab_noise_cutmix_fmix_n_over_tta.pkl,tf_efficientnet_b4_ns_fold10_tta.pkl,tf_efficientnet_b4_ns_fold3_tta.pkl,tf_efficientnet_b4_ns_tta.pkl,vit_b16_224_fold10_tta.pkl,vit_base_patch16_224.fit_tta.pkl,vit_base_patch32_384.fit_tta.pkl,22019613_efficientnet-b4.pkl,22019613_tta3_efficientnet-b4.pkl,22019629_efficientnet-b4.pkl,22019630_se_resnet101.pkl,22019630_tta3_se_resnet101.pkl,22019631_se_resnext101.pkl,22019631_tta3_se_resnext101.pkl,22019632_timm-resnest101e.pkl,22019632_tta3_timm-resnest101e.pkl,22019638_timm-regnety_032.pkl,22019638_tta3_timm-regnety_032.pkl,22019640_timm-resnest200e.pkl,22019640_tta3_timm-resnest200e.pkl,22019717_timm-resnest101e.pkl,22019717_tta3_timm-resnest101e.pkl,22019720_efficientnet-b4.pkl,22019720_tta3_efficientnet-b4.pkl,22019725_efficientnet-b4.pkl,22019725_tta3_efficientnet-b4.pkl,ex02C_efficientnet-b4.pkl,ex02C_tta3_efficientnet-b4.pkl,ex02C_tta5_efficientnet-b4.pkl,ex05C_timm-vit_base_patch16_224.pkl,ex05C_tta10_timm-vit_base_patch16_224.pkl,ex05C_tta3_timm-vit_base_patch16_224.pkl,ex05C_tta5_timm-vit_base_patch16_224.pkl,ex07C_efficientnet-b4.pkl,ex07C_tta3_efficientnet-b4.pkl,ex07C_tta5_efficientnet-b4.pkl,ex08C_timm-resnest101e.pkl,ex08C_tta3_timm-resnest101e.pkl,ex08C_tta5_timm-resnest101e.pkl,ex09C_efficientnet-b4.pkl,ex09C_tta3_efficientnet-b4.pkl,ex09C_tta5_efficientnet-b4.pkl
byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
deit_base_patch16_224_tta.pkl,0.919175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
microsoftvisionresnet50_tta.pkl,0.931500,0.924927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
resnest101e_cleanlab_noise_cutmix_tta.pkl,0.942286,0.919004,0.923908,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
resnest101e_cleanlab_tta.pkl,0.937522,0.916887,0.929480,0.945115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

# avg_ems oof

In [6]:
import itertools
from tqdm.notebook import tqdm


def check_best_comb(df, oof_list, ens_weights, model_names, n_model=5, oof_th=0.905):
    """oof_accが最大になる組み合わせを総当たりで計算
    averageの重みは正解率"""

    assert len(oof_list) == len(ens_weights)
    # assert len(oof_list)==len(model_names)

    best_id_comb = None
    tmp_ens_oof = 0.0
    idxs = list(range(len(oof_list)))
    for id_comb in tqdm(list(itertools.combinations(idxs, n_model))):

        _oof_list = [oof_list[i] for i in id_comb]
        _ens_weights = [ens_weights[i] for i in id_comb]

        oof_ensemble = np.zeros((df.shape[0], 5))
        for pre, w in zip(_oof_list, _ens_weights):
            pre = pre[:21397]
            oof_ensemble += (w / sum(_ens_weights)) * pre

        ens_oof = accuracy_score(df["label"].values, oof_ensemble.argmax(1))

        ###########################################
        ## 全組み合わせ確認
        # print("-"*100)
        # print(id_comb)
        # print(f"ens oof:    \t{round(ens_oof, 4)}" )
        # print([model_names[i] for i in id_comb])
        ###########################################

        if ens_oof > tmp_ens_oof:
            if ens_oof > tmp_ens_oof:
                tmp_ens_oof = ens_oof
                best_id_comb = id_comb
                print("-" * 100)
                print(id_comb)
                print(f"ens oof:    \t{round(ens_oof, 4)}")
                print([model_names[i] for i in id_comb])
                print(_ens_weights)

        ###########################################
        elif oof_th > 0.0 and ens_oof > oof_th:
            print("-" * 100)
            print(id_comb)
            print(f"ens oof:    \t{round(ens_oof, 4)}")
            print([model_names[i] for i in id_comb])
            print(_ens_weights)
        ###########################################

    print([model_names[i] for i in best_id_comb])
    print([ens_weights[i] for i in best_id_comb])
    print("best acc:", tmp_ens_oof)

    return best_id_comb

In [7]:
best_id_comb = check_best_comb(
    df,
    oof_list,
    ens_weights,
    model_names,
    n_model=3,
    oof_th=0.0,
)

----------------------------------------------------------------------------------------------------
(0, 1, 2)
ens oof:    	0.8986
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8866196195728373]
----------------------------------------------------------------------------------------------------
(0, 1, 3)
ens oof:    	0.9012
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8948918072627003]
----------------------------------------------------------------------------------------------------
(0, 1, 5)
ens oof:    	0.9019
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'resnest101e_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8941907744076273]
---------------------------------------------

In [8]:
best_id_comb = check_best_comb(
    df,
    oof_list,
    ens_weights,
    model_names,
    n_model=4,
    oof_th=0.0,
)

----------------------------------------------------------------------------------------------------
(0, 1, 2, 3)
ens oof:    	0.9012
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8866196195728373, 0.8948918072627003]
----------------------------------------------------------------------------------------------------
(0, 1, 2, 4)
ens oof:    	0.9017
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8866196195728373, 0.8948450717390288]
----------------------------------------------------------------------------------------------------
(0, 1, 2, 33)
ens oof:    	0.9021
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvi

In [9]:
best_id_comb = check_best_comb(
    df,
    oof_list,
    ens_weights,
    model_names,
    n_model=5,
    oof_th=0.0,
)

----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 4)
ens oof:    	0.9026
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl', 'resnest101e_cleanlab_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8866196195728373, 0.8948918072627003, 0.8948450717390288]
----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 5)
ens oof:    	0.9026
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl', 'resnest101e_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8866196195728373, 0.8948918072627003, 0.8941907744076273]
----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 10)
ens oof

In [13]:
best_id_comb = check_best_comb(
    df,
    oof_list,
    ens_weights,
    model_names,
    n_model=6,
    # oof_th=0.9055,
    oof_th=0.0,
)

----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 4, 5)
ens oof:    	0.9032
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl', 'resnest101e_cleanlab_tta.pkl', 'resnest101e_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8866196195728373, 0.8948918072627003, 0.8948450717390288, 0.8941907744076273]
----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 4, 10)
ens oof:    	0.9034
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl', 'resnest101e_cleanlab_tta.pkl', 'tf_efficientnet_b4_ns_tta.pkl']
[0.8941907744076273, 0.8853110249100341, 0.8866196195728373, 0.8948918072627003, 0.8948450717390288, 0.8924148245081086]
-------------

# avg_ems_oof + cfm

In [10]:
import itertools
from tqdm.notebook import tqdm


def check_best_comb_cfm(df, oof_list, cfms, model_names, n_model=5, oof_th=0.905):
    """oof_accが最大になる組み合わせを総当たりで計算
    averageの重みは混同行列から計算する"""

    assert len(oof_list) == len(cfms)
    # assert len(oof_list)==len(model_names)

    best_ens_weights = None
    best_id_comb = None
    tmp_ens_oof = 0.0
    idxs = list(range(len(oof_list)))
    for id_comb in tqdm(list(itertools.combinations(idxs, n_model))):

        _oof_list = [oof_list[i] for i in id_comb]
        _cfms = np.array([cfms[i] for i in id_comb])
        _cfm_diag = np.array([np.diag(_c) for _c in _cfms])  # 混同行列の対角成分取り出す
        _ens_weights = _cfm_diag / np.sum(_cfm_diag, axis=0)  # 各クラスの総和（列方向）で正規化
        # print(_cfm_diag)
        # print(_ens_weights)

        oof_ensemble = np.zeros((df.shape[0], 5))
        for pre, w in zip(_oof_list, _ens_weights):
            pre = pre[:21397]
            oof_ensemble += w * pre

        ens_oof = accuracy_score(df["label"].values, oof_ensemble.argmax(1))

        if ens_oof > tmp_ens_oof:
            print("-" * 100)
            print(id_comb)
            if ens_oof > tmp_ens_oof:
                print(f"ens oof:    \t{round(ens_oof, 4)}")
                print([model_names[i] for i in id_comb])
                print(_ens_weights)
                tmp_ens_oof = ens_oof
                best_id_comb = id_comb
                best_ens_weights = _ens_weights

        ###########################################
        elif oof_th > 0.0 and ens_oof > oof_th:
            print("-" * 100)
            print(id_comb)
            print(f"ens oof:    \t{round(ens_oof, 4)}")
            print([model_names[i] for i in id_comb])
            print(_ens_weights)
        ###########################################

    print([model_names[i] for i in best_id_comb])
    print([ens_weights[i] for i in best_id_comb])
    print("best acc:", tmp_ens_oof)
    print("best_ens_weights:\n", best_ens_weights)

    return best_id_comb, best_ens_weights

In [11]:
best_id_comb, best_ens_weights = check_best_comb_cfm(
    df,
    oof_list,
    cfms,
    model_names,
    # n_model=2,
    # n_model=3,
    # n_model=4,
    n_model=5,
    # oof_th=0.9055,
    oof_th=0.0,
)

----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 4)
ens oof:    	0.9026
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl', 'resnest101e_cleanlab_tta.pkl']
[[0.20949883 0.2003218  0.2076907  0.19994357 0.19582272]
 [0.19347319 0.19779336 0.18632764 0.20113483 0.19745288]
 [0.1958042  0.19572463 0.19810338 0.19908148 0.20315843]
 [0.20192308 0.20342489 0.20446019 0.19959873 0.20264901]
 [0.1993007  0.20273532 0.20341809 0.20024139 0.20091696]]
----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 10)
ens oof:    	0.9041
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl', 'tf_efficientnet_b4_ns_tta.pkl']
[[0.20584025 0.20064464 0

In [12]:
best_id_comb, best_ens_weights = check_best_comb_cfm(
    df,
    oof_list,
    cfms,
    model_names,
    n_model=6,
    # oof_th=0.9055,
    oof_th=0.0,
)

----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 4, 5)
ens oof:    	0.903
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_cleanlab_noise_cutmix_tta.pkl', 'resnest101e_cleanlab_tta.pkl', 'resnest101e_tta.pkl']
[[0.17180406 0.16693803 0.17316882 0.16661442 0.16315789]
 [0.15866189 0.16483095 0.15535668 0.16760711 0.16451613]
 [0.16057348 0.16310698 0.16517508 0.16589603 0.16926995]
 [0.1655914  0.16952399 0.17047528 0.16632706 0.1688455 ]
 [0.16344086 0.16894933 0.16960639 0.16686259 0.16740238]
 [0.17992832 0.1666507  0.16621774 0.16669279 0.16680815]]
----------------------------------------------------------------------------------------------------
(0, 1, 2, 3, 4, 10)
ens oof:    	0.9033
['byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl', 'deit_base_patch16_224_tta.pkl', 'microsoftvisionresnet50_tta.pkl', 'resnest101e_clea